In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [74]:
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [75]:
comments=pd.read_csv('comments_data/AnotherTop10CopycatMovies_comments.csv')
tran=pd.read_csv('comments_data/AnotherTop10CopycatMovies_transcript_.csv')

comments['Comment_length']=comments.Comment.apply(lambda s: s if type(s)==float else len(s))


In [7]:
comments[comments.Comment_length>200].head()

,Unnamed: 0,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,Comment_length
26,26,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,Ok ok ok... so you guys are saying that Megami...,Ugzkvu8XxvBCkWj7bMB4AaABAg,7,75,253
70,70,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,yes Turner and Hooch and K9 have similar story...,Ugyvu7PKsWRciakbSRR4AaABAg,1,5,249


In [83]:
comments.isnull().values.any()

False

In [88]:
all_tran_text=''
for s in tran.text:
    all_tran_text+=s+' '

"hese movies gave us an uncanny sense of deja vu welcome to watchmojo.com cat movies for this list we're looking at more twin films that share similar premises or plots and came out around the same ti"

In [101]:
comments.append({'Comment':all_tran_text }, ignore_index=True)['Comment'].tail()

96               I AM \n\n\n\n\n\n\n\nThe flying Gorilla
97     How dare you compare Megamind to the nonsense ...
98     They made a third movie this year angel has fa...
99     Project X & 21 and Over. They came out roughly...
100    these movies gave us an uncanny sense of deja ...
Name: Comment, dtype: object

In [19]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [102]:
# Might take awhile...
bow_transformer = CountVectorizer(analyzer=text_process).fit(comments.append({'Comment':all_tran_text }, ignore_index=True)['Comment'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

1087


In [125]:
comments_bow = bow_transformer.transform(comments.append({'Comment':all_tran_text }, ignore_index=True)['Comment'])

In [157]:
print(bow_transformer.transform([' Project X  21  ']))

  (0, 9)	1
  (0, 174)	1
  (0, 222)	1


In [128]:
print('Shape of Sparse Matrix: ', comments_bow.shape)
print('Amount of Non-Zero occurences: ', comments_bow.nnz)

Shape of Sparse Matrix:  (101, 1087)
Amount of Non-Zero occurences:  1448


In [129]:
sparsity = (100.0 * comments_bow.nnz / (comments_bow.shape[0] * comments_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity,3)))

sparsity: 1.319


In [130]:
tfidf_transformer = TfidfTransformer().fit(comments_bow)

In [132]:
comments_tfidf = tfidf_transformer.transform(comments_bow)
print(comments_tfidf.shape)

(101, 1087)


In [159]:
print(tfidf_transformer.transform(bow_transformer.transform([' Project X  21  '])))

  (0, 222)	0.5773502691896258
  (0, 174)	0.5773502691896258
  (0, 9)	0.5773502691896258


## testing cosine sim

In [184]:
tran_string_test=tfidf_transformer.transform(bow_transformer.transform([tran.text.iloc[200]]))
comment_string_test=tfidf_transformer.transform(bow_transformer.transform([comments.Comment.iloc[1]]))   

In [185]:
cosine_similarity(tran_string_test,comment_string_test)

array([[0.07109088]])

In [183]:
print(tran.text.iloc[200])
print(comments.Comment.iloc[1])

dreamworks Megamind arrived right on the
The biggest difference between Megamind and Despicable Me is that in Megamind the comic relief (Minion) is a million times more likeable and we can actually understand him


## create a cosine sim column for a specific tst string

In [198]:
print(tran.text.iloc[200])

dreamworks Megamind arrived right on the


In [ ]:
tran_string_test=tfidf_transformer.transform(bow_transformer.transform([tran.text.iloc[200]]))

In [192]:
comments['sim_score']=comments.Comment.apply(lambda s: cosine_similarity(tran_string_test,tfidf_transformer.transform(bow_transformer.transform([s])))[0][0] )

In [203]:
comments.sort_values(by='sim_score',ascending=False).head()

,Unnamed: 0,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,Comment_length,sim_score
60,60,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,Oh ok so John Wick is a copy of The transport...,Ugxzu_sDmB65sdPtJCR4AaABAg,0,2,108,0.144723
4,4,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,Megamind is not a Copycat of Despicable Me.,UgyTRu8Q9OuyDvm2kPd4AaABAg,18,478,43,0.097064
7,7,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,Megamind is way better than Despicable Me,Ugz2KboTZKfc_lhjixZ4AaABAg,21,552,41,0.084050
48,48,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,Megamind is WAAAY better than Despicable Me,UgzR0gWO7GNz24zsbdN4AaABAg,0,3,43,0.080619
54,54,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,I see Megamind I click,UgwXGuWPmAM3Bj46aDl4AaABAg,0,9,22,0.079262


In [215]:
print(comments.sort_values(by='sim_score',ascending=False).Comment.iloc[25])

Megamind and DESPICABLE ME are both awesome movies and HAPPY FEET and SURF'S UP are great movie


## pick out a time slot from transcript

In [256]:
tran['end']=tran['start']+tran['duration']

In [304]:
def select_script_transcript(start_time,added_time,tran):
    #start_time=20
    end_time=start_time+added_time
    #a[0] <= b[0] <= a[1] or b[0] <= a[0] <= b[1]
    #tran[((start_time<=tran['start'])&(tran['start']<=end_time))]
    #tran[((tran['start']<=start_time)&(start_time<=tran['end']))]

    transelection=tran[((start_time<=tran['start'])&(tran['start']<=end_time)) | ((tran['start']<=start_time)&(start_time<=tran['end']))]

    transelection_text=''
    for s in transelection.text:
        transelection_text+=s+' '
    return(transelection_text)

In [305]:
select_script_transcript(20,10,tran)

'the same time a few of these even had the same endings so beware spoilers ahead number 10 flight plan and redeye '

In [296]:
tran.head(9)

,Unnamed: 0,text,start,duration,end
0,0,these movies gave us an uncanny sense of,0.680,9.220,9.900
1,1,deja vu welcome to watchmojo.com cat,3.810,8.459,12.269
2,2,movies for this list we're looking at,9.900,4.380,14.280
3,3,more twin films that share similar,12.269,4.680,16.949
4,4,premises or plots and came out around,14.280,5.070,19.350
5,5,the same time a few of these even had,16.949,4.981,21.930
6,6,the same endings so beware spoilers,19.350,12.179,31.529
7,7,ahead number 10 flight plan and redeye,21.930,12.899,34.829
8,8,in both these high-altitude thrillers a,31.529,5.701,37.230


In [278]:
def overlap(a,b):
    return a[0] <= b[0] <= a[1] or b[0] <= a[0] <= b[1]

In [280]:
overlap([0,5],[102,400])

False

In [34]:
# def get_cosine_sim(vectors): 
#     vectors = [t for t in vectors]
#     return cosine_similarity(vectors)

In [43]:
cosine_similarity(np.reshape([1,0,3],(3,1)),np.reshape([0,1,0],(3,1)))

array([[0., 1., 0.],
       [0., 0., 0.],
       [0., 1., 0.]])

In [49]:
np.reshape([1,0,3],(3,1))


array([[1],
       [0],
       [3]])

In [44]:
[1,0,3]

[1, 0, 3]

In [63]:
cosine_similarity([[1,2,3]],[[0,0,1]])

array([[0.80178373]])

In [178]:
'jesse'.find('s')

2

In [182]:
tran[tran.text.str.contains('Megamind')]

,Unnamed: 0,text,start,duration
200,200,dreamworks Megamind arrived right on the,602.110,4.68
212,212,orphan children Megamind on the other,640.649,4.65
